In [1]:
#import dependencies
import pandas as pd
import gmaps
import requests

#import api key
from config import g_key

In [2]:
#store csv saved from weatherpy into df
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mar Del Plata,AR,2021-06-26 20:15:42,-38.0023,-57.5575,11.67,60,90,1.34
1,1,Jamestown,US,2021-06-26 20:12:43,42.0970,-79.2353,29.62,61,90,4.02
2,2,Dublin,US,2021-06-26 20:15:21,37.7021,-121.9358,31.88,47,1,4.47
3,3,Laguna,US,2021-06-26 20:11:16,38.4210,-121.4238,38.47,39,1,0.89
4,4,Tazovskiy,RU,2021-06-26 20:15:49,67.4667,78.7000,4.92,71,0,4.57


In [3]:
#get dtypes
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
#configure gmaps to use Google API key
gmaps.configure(api_key=g_key)

## Temperature Heatmap

In [5]:
#get lat and lon
locations = city_data_df[['Lat', 'Lng']]
#get max temp
max_temp = city_data_df['Max Temp']
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
#assign figure variable
fig = gmaps.figure(center = (30.0,31.0), zoom_level = 1.5)
#assign heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating = False,
                                 max_intensity = 300, point_radius=4)
#add heatmap layer
fig.add_layer(heat_layer)
#call figure to plot data
fig

Figure(layout=FigureLayout(height='420px'))

## Percent Humidity Heatmap

In [6]:
# Heatmap of % Humidity
locations = city_data_df[['Lat', 'Lng']]
humidity = city_data_df['Humidity']
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, 
                              max_intensity = 300, point_radius = 4)

fig.add_layer(heat_layer)
#call fig to plot data
fig

Figure(layout=FigureLayout(height='420px'))

## Percent Cloudiness Heatmap

In [7]:
# Heatmap of % cloudiness
locations = city_data_df[['Lat', 'Lng']]
clouds = city_data_df['Cloudiness']
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations, weights = clouds,
                               dissipating = False, max_intensity = 300,
                               point_radius = 4)

fig.add_layer(heat_layer)

#call fig to plot data
fig

Figure(layout=FigureLayout(height='420px'))

## Wind Speed Heatmap

In [8]:
# Heatmap of % cloudiness
locations = city_data_df[['Lat', 'Lng']]
winds = city_data_df['Wind Speed']
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations, weights = winds,
                               dissipating = False, max_intensity = 300,
                               point_radius = 4)

fig.add_layer(heat_layer)

#call fig to plot data
fig

Figure(layout=FigureLayout(height='420px'))

## Vacation Criteria

In [9]:
#ask customer to add min and max temp value
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

#if statement in case somebody inputs the temps backwards
if min_temp > max_temp:
    temp = max_temp
    max_temp = min_temp
    min_temp = temp

What is the minimum temperature you would like for your trip? 0
What is the maximum temperature you would like for your trip? 25


In [10]:
#filter dataset to find the cities 
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mar Del Plata,AR,2021-06-26 20:15:42,-38.0023,-57.5575,11.67,60,90,1.34
4,4,Tazovskiy,RU,2021-06-26 20:15:49,67.4667,78.7000,4.92,71,0,4.57
5,5,Dikson,RU,2021-06-26 20:15:50,73.5069,80.5464,0.18,87,20,1.00
7,7,Arraial Do Cabo,BR,2021-06-26 20:11:13,-22.9661,-42.0278,22.92,73,0,7.20
8,8,Cape Town,ZA,2021-06-26 20:12:38,-33.9258,18.4232,14.08,75,100,0.89
9,9,Morondava,MG,2021-06-26 20:15:56,-20.2833,44.2833,19.73,68,0,3.62
10,10,Port Alfred,ZA,2021-06-26 20:15:57,-33.5906,26.8910,15.02,69,27,9.28
11,11,Ostersund,SE,2021-06-26 20:15:58,63.1792,14.6357,16.45,67,20,5.66
12,12,Saint-Philippe,RE,2021-06-26 20:16:00,-21.3585,55.7679,19.77,80,26,3.90
13,13,Talnakh,RU,2021-06-26 20:16:02,69.4865,88.3972,8.25,66,100,3.90


In [11]:
preferred_cities_df.count()

City_ID       340
City          340
Country       334
Date          340
Lat           340
Lng           340
Max Temp      340
Humidity      340
Cloudiness    340
Wind Speed    340
dtype: int64

### Mapping and Hotel Search

In [12]:
#create df to store hotel names
hotel_df = preferred_cities_df[['City', 'Country', 'Max Temp', 'Lat', 'Lng']]
hotel_df['Hotel Name'] = ''
hotel_df.head(10)

C:\Users\Ranita\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Mar Del Plata,AR,11.67,-38.0023,-57.5575,
4,Tazovskiy,RU,4.92,67.4667,78.7000,
5,Dikson,RU,0.18,73.5069,80.5464,
7,Arraial Do Cabo,BR,22.92,-22.9661,-42.0278,
8,Cape Town,ZA,14.08,-33.9258,18.4232,
9,Morondava,MG,19.73,-20.2833,44.2833,
10,Port Alfred,ZA,15.02,-33.5906,26.8910,
11,Ostersund,SE,16.45,63.1792,14.6357,
12,Saint-Philippe,RE,19.77,-21.3585,55.7679,
13,Talnakh,RU,8.25,69.4865,88.3972,


In [13]:
# set patameters to search for hotel
params = {
    'radius':5000,
    'type':'lodging',
    'key':g_key
}

In [14]:
#iterate through the df
for index, row in hotel_df.iterrows():
    #Get lat and lon
    lat = row['Lat']
    lng = row['Lng']
    
    #add coords to location key for params dict
    params['location'] = f'{lat},{lng}'
    
    #use search term 'lodging' amd our lat and lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #make request and get JSON data from search
    hotels = requests.get(base_url, params=params).json()
    #grab 1st hotel from results and store name
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except(IndexError):
        print('Hotel not found... skipping.')

hotel_df.head()

C:\Users\Ranita\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Mar Del Plata,AR,11.67,-38.0023,-57.5575,Gran Hotel Mar del Plata
4,Tazovskiy,RU,4.92,67.4667,78.7000,TazHostel
5,Dikson,RU,0.18,73.5069,80.5464,
7,Arraial Do Cabo,BR,22.92,-22.9661,-42.0278,Pousada Porto Praia
8,Cape Town,ZA,14.08,-33.9258,18.4232,Southern Sun Waterfront Cape Town


## Heatmap from Hotel DataFrame

In [21]:
#pop-up markers for info on hotels
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °C</dd>
</dl>
"""

In [22]:
#Store df Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [23]:
#add a heatmap of temperature for vacation spots
locations = hotel_df[['Lat', 'Lng']]
max_temp = hotel_df['Max Temp']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = max_temp, dissipating = False, 
                                max_intensity = 300, point_radius = 4)
#adding markers
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
#call fig to plot data
fig

Figure(layout=FigureLayout(height='420px'))